In [1]:
pip install spotipy pandas jupyter


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/255.6 kB ? eta -:--:--
   --------- ------------------------------ 61.4/255.6 kB 3.2 MB/s eta 0:00:01
   ------------------------------ --------- 194.6/255.6 kB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 255.6/255.6 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Import necessary libraries
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Set up your Spotify API credentials
CLIENT_ID = 'Your_clientID'
CLIENT_SECRET = 'Your_Client_secret'
REDIRECT_URI = 'http://localhost:8501'

# Initialize the Spotify API client
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                               client_secret=CLIENT_SECRET,
                                               redirect_uri=REDIRECT_URI,
                                               scope='user-library-read'))

def get_user_tracks():
    results = sp.current_user_saved_tracks()
    tracks = results['items']
    track_list = []
    for item in tracks:
        track = item['track']
        track_list.append({
            'name': track['name'],
            'artist': track['artists'][0]['name'],
            'album': track['album']['name'],
            'uri': track['uri']
        })
    return pd.DataFrame(track_list)

# Fetch and display user tracks
user_tracks = get_user_tracks()
user_tracks.head()  # Display the first few tracks


# Function to recommend songs based on user preferences and mood
def recommend_songs(preferred_genre, preferred_artist, preferred_mood):
    mood_genre_mapping = {
        "Happy": "Pop",
        "Sad": "Ballad",
        "Chill": "Chill",
        "Energetic": "Dance",
        "Romantic": "Romance",
        "Motivational": "Rock",
        "Melancholic": "Jazz"
    }
    
    recommended_tracks = []
    genre = mood_genre_mapping.get(preferred_mood, preferred_genre)

    # Create a search query based on the selected genre and mood
    if preferred_artist:
        query = f'genre:{genre} artist:{preferred_artist}'
    else:
        query = f'genre:{genre}'

    # Fetch related tracks based on the query
    results = sp.search(q=query, type='track', limit=10)

    # Collect recommended tracks
    for track in results['tracks']['items']:
        recommended_tracks.append({
            'name': track['name'],
            'artist': track['artists'][0]['name'],
            'album': track['album']['name'],
            'uri': track['uri']
        })

    # Convert the list of recommended tracks to a DataFrame
    recommended_tracks_df = pd.DataFrame(recommended_tracks)

    # If no recommendations found, provide fallback options
    if recommended_tracks_df.empty:
        print("No recommendations found based on your input. Here are some popular tracks from the selected genre:")
        fallback_results = sp.search(q=f'genre:{genre}', type='track', limit=10)
        fallback_tracks = []
        for track in fallback_results['tracks']['items']:
            fallback_tracks.append({
                'name': track['name'],
                'artist': track['artists'][0]['name'],
                'album': track['album']['name'],
                'uri': track['uri']
            })
        # Convert fallback tracks to DataFrame
        recommended_tracks_df = pd.DataFrame(fallback_tracks)

    return recommended_tracks_df


In [ ]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Set up your Spotify API credentials
CLIENT_ID = 'your-client-id'
CLIENT_SECRET = 'your-client-secret'
REDIRECT_URI = 'http://localhost:8501'

# Initialize the Spotify API client
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                               client_secret=CLIENT_SECRET,
                                               redirect_uri=REDIRECT_URI,
                                               scope='user-library-read'))

# Function to fetch user's saved tracks
def get_user_tracks():
    results = sp.current_user_saved_tracks()
    tracks = results['items']
    track_list = []
    for item in tracks:
        track = item['track']
        track_list.append({
            'name': track['name'],
            'artist': track['artists'][0]['name'],
            'album': track['album']['name'],
            'uri': track['uri']
        })
    return pd.DataFrame(track_list)

# Function to recommend tracks based on user's tracks
def recommend_tracks(user_tracks):
    unique_artists = user_tracks['artist'].unique()
    
    recommended_tracks = []
    for artist in unique_artists:
        results = sp.search(q='artist:' + artist, type='track', limit=5)
        for track in results['tracks']['items']:
            recommended_tracks.append({
                'name': track['name'],
                'artist': track['artists'][0]['name'],
                'album': track['album']['name'],
                'uri': track['uri']
            })
    return pd.DataFrame(recommended_tracks)

# Function to simulate ground truth data (for demonstration)
def get_ground_truth():
    # This should be your actual ground truth data for testing
    return pd.DataFrame({
        'name': ['Song A', 'Song B', 'Song C'],
        'artist': ['Artist 1', 'Artist 2', 'Artist 3'],
        'album': ['Album X', 'Album Y', 'Album Z'],
        'uri': ['uri1', 'uri2', 'uri3']
    })

# Fetch user tracks and recommendations
user_tracks = get_user_tracks()
recommended_tracks = recommend_tracks(user_tracks)

# Get ground truth data (known correct recommendations)
ground_truth = get_ground_truth()

# Convert to a common format to compare (e.g., URIs for simplicity)
recommended_uris = set(recommended_tracks['uri'])
ground_truth_uris = set(ground_truth['uri'])

# Create lists for evaluation
y_true = [1 if uri in ground_truth_uris else 0 for uri in recommended_uris]
y_pred = [1] * len(recommended_uris)  # All recommended items are considered positive

# Calculate precision, recall, and F1 score
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
